In [3]:
import numpy as np
import os
import cv2
from math import sqrt

In [4]:
def extract_data(path):
    X_train=[]
    y_train=[]
    X_test=[]
    y_test=[]

    file=os.listdir(path)

    for entity in file:
        try:
            entityPath=os.listdir(f"{path}/{entity}")
            for i,data in enumerate(entityPath):
                try:
                    current_img=cv2.imread(f"{path}/{entity}/{data}")
                    if current_img is not None:
                        if ((len(entityPath)>1) and (i==0)):
                            X_test.append(current_img)
                            y_test.append(entity)
                        else:
                            X_train.append(current_img)
                            y_train.append(entity)
                    else:
                        print(f"{path}/{entity}/{data} is None type")
                except Exception as e:
                    print(f"Erreur avec {path}/{entity}/{data} : {e}")
        except Exception as ex:
            print(f"Erreur avec {path}/{entity} : {ex}")
    
    return X_train,y_train,X_test,y_test

In [5]:
def imgProcess(img):
    if not isinstance(img,np.ndarray):
        print("convertion de l'img")
        img=cv2.imread(img)
    gris=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #RGB -> Gray -> dim(255,255,3)=255*255*3 val de couleur = 195075 vals -> dim(255,255,1) -> 255*255*1 val de couleur = 65025 vals pour l'exemple du 255*255
    resize=cv2.resize(gris,(100,100)) #dim -> dim(100,100,1) -> 100*100*1 = 10000 val de couleur
    flat=resize.flatten() #2D -> 1D vec de (10000,1)
    return flat

In [6]:
def distEuclid(img1,img2): #p1(a1,a2,a3...,a10000) p2(b1,b2,b3...,b10000) car chaque image a 10000 val de couleur
    #img1 = img1.astype(np.int32) #pour avoir des valeur négatives sinon en uint8 c'est de 0 à 255 (pas negatif)
    #img2 = img2.astype(np.int32)
    diff=img1.astype(np.int32)-img2.astype(np.int32) #opti

    #distance euclidienne entre deux points (en 2D) p1(x1,y1) et p2(x2,y2): d=hypothénuse du triangle rectangle
    #ainsi d²=a²+b² et d=sqrt(a²+b²) avec a=abs(x1-x2) et b=abs(y1-y2) DONC -> d=sqrt((x1-x2)²+(y1-y2)²)
    #en n dim : d=sqrt((a1-b1)²+(a2-b2)²...+(an-bn)²) = sqrt(sum[1;N]((a-b)²))
    #return sqrt(sum((a-b)**2 for a,b in zip(img1,img2))) #zip -> pour deux listes
    return np.sqrt(np.sum(diff**2)) #opti

In [7]:
def knn_predict(X_train,y_train, newImg,k):
    distances=[]
    for i in range(len(X_train)):
        dist=distEuclid(X_train[i],newImg) #on calcule la distance entre la nouvelle image et toutes les autres images
        distances.append((dist,y_train[i])) #on stock la distance avec le label de chaque image à laquelle on a calculé la distance
    
    distances.sort(key=lambda x: x[0]) #on trie la liste de distances dans l'ordre croissant comme ça les derniers termes sont les plus proche de la nouvelle image
    k_voisins=distances[:k] #on prend les k plus proches voisins

    counts={} #dictionnaire pour compter les labels les plus proches et leur occurences
    for _,label in k_voisins: #dans le tableau c'est [distance,label], ici on veut que le label donc _
        counts[label]=counts.get(label,0)+1 #.get(a,b) = prend la val a si existe sinon prend la val b

    return max(counts,key=counts.get) #compare les valeurs mais renvoie le label du dictionnaire

In [8]:
def knn_evaluate(processed_train,y_train,processed_test,y_test,k,n):
    knn_results=[]
    true=0
    for i,img in enumerate(processed_test[:n]):
        print(f"Image KNN : {i}")
        knn_results.append(knn_predict(processed_train,y_train,processed_test[i],k))
        if knn_results[i]==y_test[i]:
            true+=1
    print(f"Accuracy (%) : {100*true/n}")

    return knn_results

In [9]:
def KNN(X_train,y_train,X_test,y_test,k=3,n=10):
    processed_train=[]
    for img in X_train:
        processed_train.append(imgProcess(img))

    processed_test=[]
    for img in X_test:
        processed_test.append(imgProcess(img))

    knn_result=knn_evaluate(processed_train,y_train,processed_test,y_test,k,n)

In [10]:
def imgPadding(img,padding):
    size=img.shape[0]
    if padding is not None:
        size=size+2*padding
        newImg=np.zeros((size,size,3),dtype=np.uint8)
        for x in range(padding,size-padding):
            for y in range(padding,size-padding):
                newImg[x][y]=img[x-padding][y-padding]
    else:
        newImg=img
    return newImg

In [27]:
def max_pooling(img,dim=2,padding=None,stride=1):
    if padding is not None:
        img=imgPadding(img,padding)
    
    newDim=((img.shape[0]-dim)//stride)+1
    newImg=np.zeros([newDim,newDim,3])
    
    for rgb in range(img.shape[2]):
        for x in range(0,(img.shape[0]//dim)*dim,stride):
            for y in range(0,(img.shape[0]//dim)*dim,stride):
                max=0
                for h in range(dim):
                    for l in range(dim):
                        if img[x+l][y+h][rgb]>max:
                            max=img[x+l][y+h][rgb]
                newImg[((x-dim)//stride)+1][((y-dim)//stride)+1][rgb]=max
    
    return newImg

In [32]:
def average_pooling(img,dim=2,padding=None,stride=1):
    if padding is not None:
        img=imgPadding(img,padding)
    
    newDim=((img.shape[0]-dim)//stride)+1
    newImg=np.zeros([newDim,newDim,3])
    
    for rgb in range(img.shape[2]):
        for x in range(0,(img.shape[0]//dim)*dim,stride):
            for y in range(0,(img.shape[0]//dim)*dim,stride):
                somme=0
                for h in range(dim):
                    for l in range(dim):
                        somme+=img[x+l][y+h][rgb]
                newImg[((x-dim)//stride)+1][((y-dim)//stride)+1][rgb]=somme//dim*dim
    
    return newImg

In [ ]:
X_train,y_train,X_test,y_test=extract_data("Faces")
knn_results=KNN(X_train,y_train,X_test,y_test)

Erreur avec Faces/pairs.txt : [Errno 20] Not a directory: 'Faces/pairs.txt'
Erreur avec Faces/pairs_01.txt : [Errno 20] Not a directory: 'Faces/pairs_01.txt'
Erreur avec Faces/pairs_02.txt : [Errno 20] Not a directory: 'Faces/pairs_02.txt'
Erreur avec Faces/pairs_03.txt : [Errno 20] Not a directory: 'Faces/pairs_03.txt'
Erreur avec Faces/pairs_04.txt : [Errno 20] Not a directory: 'Faces/pairs_04.txt'
Erreur avec Faces/pairs_05.txt : [Errno 20] Not a directory: 'Faces/pairs_05.txt'
Erreur avec Faces/pairs_06.txt : [Errno 20] Not a directory: 'Faces/pairs_06.txt'
Erreur avec Faces/pairs_07.txt : [Errno 20] Not a directory: 'Faces/pairs_07.txt'
Erreur avec Faces/pairs_08.txt : [Errno 20] Not a directory: 'Faces/pairs_08.txt'
Erreur avec Faces/pairs_09.txt : [Errno 20] Not a directory: 'Faces/pairs_09.txt'
Erreur avec Faces/pairs_10.txt : [Errno 20] Not a directory: 'Faces/pairs_10.txt'
Image KNN : 0
Image KNN : 1
Image KNN : 2
Image KNN : 3
Image KNN : 4
Image KNN : 5
Image KNN : 6
Image 